In [1]:
from pathlib import Path
import json 

p_ann = Path("/data/soyeonhong/nlq/nlq_lightning/NLQ-query-uniner-extracted-object/NLQ-query-uniner-extracted-object.json")
p_ann_train = Path("/data/soyeonhong/nlq/nlq_lightning/data/unified/annotations.NLQ_train.json")
p_ann_val = Path("/data/soyeonhong/nlq/nlq_lightning/data/unified/annotations.NLQ_val.json")
anns = json.loads(p_ann.read_text())
ann_train = json.loads(p_ann_train.read_text())
ann_val = json.loads(p_ann_val.read_text())

len(anns), len(ann_train), len(ann_val)


FileNotFoundError: [Errno 2] No such file or directory: '/data/soyeonhong/nlq/nlq_lightning/NLQ-query-uniner-extracted-object/NLQ-query-uniner-extracted-object.json'

In [8]:
anns[0]

{'query': 'Where did I saw fruits?',
 'query_id': '40dbe5c4-67f1-47fa-bf10-0c4462668c74_5',
 'start': 388.62356,
 'end': 389.759,
 'clip_uid': '708787f2-2e7f-4988-8cea-e280eec35f3f',
 'template': 'Place: Where did I put X?',
 'entities': {'type': 'object', 'values': ['fruits']}}

In [12]:
new_data = []
for ann_gt in ann_val:
    for ann in anns:
        if ann_gt['sample_id'] == ann['query_id']:
            ann_gt['entities'] = ann['entities']
            new_data.append(ann_gt)
            
p_new = Path("/data/soyeonhong/nlq/nlq_lightning/data/unified/annotations.NLQ_val_object.json")
p_new.write_text(json.dumps(new_data))

1685578

In [3]:
p_llava = Path("/data/soyeonhong/llm/universal-ner/LLaVA-NeXT-Video-7B-DPO-uniner-extracted/version-1")
file_lists = list(p_llava.glob("*.json"))


obj_all = []

for file in file_lists:
    results = json.loads(file.read_text())
    for result in results:
        objs = result['entities']['values']
        
        for obj in objs:
            if not obj in obj_all and obj != "":    
                obj_all.append(obj)


In [8]:
for ann in anns:
    objs = ann['entities']['values']
    
    for obj in objs:
        if not obj in obj_all and obj != "":
            obj_all.append(obj)

In [9]:

p_obj = Path("/data/soyeonhong/nlq/nlq_lightning/data/unified/objects.json")
p_obj.write_text(json.dumps(obj_all))

1469867

In [22]:
import torch


v_feat = torch.rand([6, 900, 768])
v_mask = torch.rand([6, 900])
slice_tmp = torch.tensor([[0, 10], [10, 20], [20, 30], [30, 40], [40, 50], [50, 60]])

tmp = torch.stack([v_feat[i, slice[0]:slice[1]] for i, slice in enumerate(slice_tmp)])
tmp.shape

torch.Size([6, 10, 768])

In [6]:
from pathlib import Path 
import json

p_ann = Path("/data/soyeonhong/nlq/nlq_lightning/data/unified/annotations.NLQ_train.json")
anns = json.loads(p_ann.read_text())
time_per_v_feat = 0.53
slice_int = 10

for ann in anns:
    clip_duration = ann['clip_duration']
    start_time = ann['clip_start_sec']
    end_time = ann['clip_end_sec']
    center = (start_time + end_time) / 2
    slice_t_s = max(0, center - slice_int / 2)
    slice_t_e = min(clip_duration, center + slice_int / 2)  
    
    
    if slice_t_s == 0:
        slice_idx_s = 0
        slice_idx_e = slice_int // time_per_v_feat
    elif slice_t_e == clip_duration:
        slice_idx_e = clip_duration / 0.53 - 1
        slice_idx_s = slice_idx_e - slice_int // time_per_v_feat
    else:
        center_idx = center / time_per_v_feat
        slice_idx_s = center_idx - slice_int // 2 // time_per_v_feat
        slice_idx_e = center_idx + slice_int // 2 // time_per_v_feat  
        
    if slice_idx_e - slice_idx_s <=12:        
        
        print(slice_idx_s, slice_idx_e, slice_idx_e - slice_idx_s)